In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import selenium
from selenium import webdriver

In [2]:
result = requests.get('http://www.ifaq.gov.sg/pdpc/apps/fcd_faqmain.aspx')
soup = BeautifulSoup(result.content)

In [120]:
main_tree = soup.find_all('a', attrs={'class':'tree_childlink'})
qa_links = [item.get('href') for item in main_tree]

In [135]:
def get_meta(qna_section):
    meta_list = qna_section.find_all(['h2', 'h3'])
    meta_list = [item.text for item in meta_list]
    meta = ', '.join(meta_list)
    return meta+'\n'

def get_questions(qna_section):
    question_list = qna_section.find_all('a', attrs={'title':'View answer'})
    question_list = [item.text.replace('\r', '').replace('\n', '') for item in question_list]
    return question_list

def get_answers(qna_section):
    answer_list = qna_section.find_all('div', attrs={'class':'faq_answer'})
    answer_list = [re.sub(r'\n+', '\n', item.text.replace('\r', '').replace('\xa0', '')).strip() for item in answer_list]
    return answer_list

def get_qa_pair(link):
    soup = BeautifulSoup(requests.get(link).content)
    qna_section = soup.find('div', attrs={'id':'print'})
    meta_string = get_meta(qna_section)
    question_list = get_questions(qna_section)
    answer_list = get_answers(qna_section)
    assert len(question_list)==len(answer_list), 'questions and answers not equal {}, {}'.format(len(question_list), len(answer_list))
    df_scraped = pd.DataFrame({'question':question_list, 'meta':[meta_string]*len(question_list), 'answer':answer_list})
    return df_scraped


In [142]:
df_pdpa = pd.DataFrame(columns=['question', 'meta', 'answer'])
for qa_link in qa_links:
    print(qa_link)
    df_scraped = get_qa_pair(qa_link)
    df_pdpa = df_pdpa.append(df_scraped, ignore_index=True)

http://www.ifaq.gov.sg:80/pdpc/apps/fcd_faqmain.aspx?qst=hRhkP9BzcBImsx2TBbssMsxu7lqt6UJK70a1wAEVmyfiNHrkSWapUoxJrod9%2bbyLC2869whY9PmzDAqIf0khkfdKfkXyUc2p%2bZeN8PHR1T32xrVks%2bZ10FLHQAia7aJUK9mcKZAJ3Hp9lWeoJh5J0jJig821hiii2%2f4Ht%2fkXC1KD6JR4U2%2fjP6hQepW%2f2liM%2fUUMs6i%2fiSeqjrqajoKZ3jq5eY14VOuQ2vBlq6ADR3g%3d
http://www.ifaq.gov.sg:80/pdpc/apps/fcd_faqmain.aspx?qst=hRhkP9BzcBImsx2TBbssMsxu7lqt6UJK70a1wAEVmyfiNHrkSWapUoxJrod9%2bbyLC2869whY9PmzDAqIf0khkfdKfkXyUc2p%2bZeN8PHR1T32xrVks%2bZ10FLHQAia7aJU%2bCgyFXiviMpCWSxFt279TDLEvkR6Q8b5MYDxklzQtgSdzGy58RgwlAsxC7JrrTOiGJPCpr6osu0NmQ1ygidvQHt4GNFv9W6FFLQQGwQbg3g%3d
http://www.ifaq.gov.sg:80/pdpc/apps/fcd_faqmain.aspx?qst=hRhkP9BzcBImsx2TBbssMsxu7lqt6UJK70a1wAEVmyfiNHrkSWapUoxJrod9%2bbyLC2869whY9PmzDAqIf0khkfdKfkXyUc2p%2bZeN8PHR1T32xrVks%2bZ10FLHQAia7aJUHAE%2fS8%2bYogJ8IO0heUp4rD%2bZ6Hb489%2f3ROOITVC9L4KaIDM9LYlQJIhMbCLr9cq31i%2fXh4RlILFKxle1sJWNpg02l8EC6PwYnEd3iU6sUiY%3d
http://www.ifaq.gov.sg:80/pdpc/apps/fcd_faqmain.aspx?qst=hRhkP9BzcBIms

In [143]:
print(len(df_pdpa))
df_pdpa

269


,question,meta,answer
0,What is personal data?,"Organisations, General\n","Personal data refers to data, whether true or ..."
1,When did the PDPA come into force?,"Organisations, General\n",The PDPA was implemented in phases to allow ti...
2,What are the objectives of the PDPA?,"Organisations, General\n",The PDPA aims to safeguard individuals’ person...
3,How does the PDPA benefit business?,"Organisations, General\n",The PDPA will strengthen Singapore's overall e...
4,How will the PDPA impact business costs?,"Organisations, General\n",The provisions of the PDPA were formulated kee...
5,How is the PDPA different from the Spam Contro...,"Organisations, General\n",The Spam Control Act (SCA) sets out a framewor...
6,What is 'publicly available'?,"Organisations, General\n",‘Publicly available’ in relation to personal d...
7,"Can an organisation collect, use or disclose p...","Organisations, General\n",While an organisation may not obtain consent f...
8,Are public databases considered publicly avail...,"Organisations, General\n",Publicly available personal data refers to per...
9,Are randomly generated data considered persona...,"Organisations, General\n","Personal data is defined under the PDPA as ""da..."


In [144]:
df_pdpa.to_csv('../data/pdpa.csv', index=False)

# Scrape legislation
It is weirdly protected, so I had to copy the html out.

In [65]:
# result = requests.get('https://sso.agc.gov.sg/Act/PDPA2012')
f = open('../data/pdpa.html', 'r', encoding='utf-8')
soup = BeautifulSoup(f.read())

In [78]:
all_parts = soup.find_all('td', attrs={'class':'part'})

In [68]:
soup.find_all('td', attrs={'class':'partHdr'})

[<td class="partHdr" id="P1I-he-">PRELIMINARY</td>,
 <td class="partHdr" id="P1II-he-">PERSONAL DATA PROTECTION COMMISSION <br/>AND ADMINISTRATION</td>,
 <td class="partHdr" id="P1III-he-">GENERAL RULES WITH RESPECT TO <br/>PROTECTION OF PERSONAL DATA</td>,
 <td class="partHdr" id="P1IV-he-">COLLECTION, USE AND DISCLOSURE OF <br/>PERSONAL DATA</td>,
 <td class="partHdr" id="P1V-he-">ACCESS TO AND CORRECTION OF <br/>PERSONAL DATA</td>,
 <td class="partHdr" id="P1VI-he-">CARE OF PERSONAL DATA</td>,
 <td class="partHdr" id="P1VII-he-">ENFORCEMENT OF PARTS III TO VI</td>,
 <td class="partHdr" id="P1VIII-he-">APPEALS TO DATA PROTECTION APPEAL COMMITTEE, HIGH COURT AND COURT OF APPEAL</td>,
 <td class="partHdr" id="P1IX-he-">DO NOT CALL REGISTRY</td>,
 <td class="partHdr" id="P1X-he-">GENERAL</td>]

In [73]:
print(len(soup.find_all('td', class_='prov1Hdr')))
print(len(soup.find_all('td', class_='prov1Txt')))

68
68


In [105]:
df_pdpa_legis = pd.DataFrame(columns=['meta', 'answer'])
for part in all_parts[1:]:
    part_header = part.find('td', class_='partHdr').text
    
    provisions = part.find_all('td', class_='prov1Hdr')
    provisions = [item.text for item in provisions]
    prov_texts = part.find_all('td', class_='prov1Txt')
    prov_texts = [re.sub(r'\n+', '\n', item.text.replace('\r', '').replace('\xa0', '')).strip() for item in prov_texts]
    print(len(prov_texts), len(provisions))
    meta_list = [part_header+', '+item for item in provisions]
    print(meta_list)
    df_part = pd.DataFrame({'meta':meta_list, 'answer':prov_texts})
    df_pdpa_legis = df_pdpa_legis.append(df_part, ignore_index=True)


6 6
['PERSONAL DATA PROTECTION COMMISSION AND ADMINISTRATION, Personal Data Protection Commission', 'PERSONAL DATA PROTECTION COMMISSION AND ADMINISTRATION, Functions of Commission', 'PERSONAL DATA PROTECTION COMMISSION AND ADMINISTRATION, Advisory committees', 'PERSONAL DATA PROTECTION COMMISSION AND ADMINISTRATION, Delegation', 'PERSONAL DATA PROTECTION COMMISSION AND ADMINISTRATION, Conduct of proceedings', 'PERSONAL DATA PROTECTION COMMISSION AND ADMINISTRATION, Co-operation agreements']
2 2
['GENERAL RULES WITH RESPECT TO PROTECTION OF PERSONAL DATA, Compliance with Act', 'GENERAL RULES WITH RESPECT TO PROTECTION OF PERSONAL DATA, Policies and practices']
8 8
['COLLECTION, USE AND DISCLOSURE OF PERSONAL DATA, Consent required', 'COLLECTION, USE AND DISCLOSURE OF PERSONAL DATA, Provision of consent', 'COLLECTION, USE AND DISCLOSURE OF PERSONAL DATA, Deemed consent', 'COLLECTION, USE AND DISCLOSURE OF PERSONAL DATA, Withdrawal of consent', 'COLLECTION, USE AND DISCLOSURE OF PERSONAL

In [111]:
len(df_pdpa_legis)
df_pdpa_legis.to_csv('../data/pdpa_legis.csv', index=False)

# Scrape ncov FAQ

In [2]:
url = 'https://www.moh.gov.sg/2019-ncov-wuhan/faqs'

In [3]:
def browser(url):
    driver = webdriver.Chrome("C:/Users/likkh/Downloads/chromedriver_win32/chromedriver.exe")
    driver.get(url)
    return driver

In [5]:
driver = browser(url)
els = driver.find_elements_by_class_name('expand')
for el in els:
    el.click()

In [6]:
soup = BeautifulSoup(driver.page_source)

In [7]:

questions=[]
answers=[]

divs = soup.find_all(attrs={'class':'show-more'})
for div in divs:
    rows = div.find_all('tr')
    for ii, row in enumerate(rows):
        texts = row.find_all('p')
        try:
            if ii%2:
                answers.append(texts[1].text)
            else:
                questions.append(texts[1].text)
        except:pass

df_ncov = pd.DataFrame({'question':questions, 'meta':['']*len(answers),'answer':answers})

In [9]:
['']*5

['', '', '', '', '']

In [10]:
df_ncov.to_csv('../data/ncov.csv', index=False)